# 对抗攻击

### 导入需要的包

In [1]:
import utils
import attack
import torch
from torch.utils.tensorboard import SummaryWriter

### 选择待评估模型与数据集

In [2]:
fig_num = 5000

path = r'./results/CIFAR10/pre-resnet-bs128-lr0.01-seed0/models/pre-resnet-bs128-lr0.01-seed0'
model = utils.creat_model('pre-resnet', path, in_ch=3, pretrained=True)
test_data = utils.data_iter('CIFAR10', 'test', fig_num, 0)[1]
imgs, labels = next(iter(test_data))
metrics = {}
input()

Files already downloaded and verified
Files already downloaded and verified


''

### 测试无对抗攻击时的模型参数

In [ ]:
top1_acc = utils.test_accuracy(model, test_imgs)

